# Выполнил Кирилл Сетдеков

In [1]:
import numpy as np
import pandas as pd
import scipy
from scipy import stats
from statsmodels.stats.weightstats import *
import matplotlib.pyplot as plt
from statsmodels.stats.descriptivestats import sign_test

import statsmodels.formula.api as smf
from sklearn.impute import SimpleImputer
import seaborn as sns
from sklearn.preprocessing import PowerTransformer


import statsmodels.api as sm
import warnings
import matplotlib
import matplotlib.dates as mdates
from pandas.plotting import lag_plot

In [2]:
%matplotlib inline
# %matplotlib widget

#  Задание 1

1. Автокорреляционная функция и частная автокорреляционная функция.

    * Автокорреляционная функция - функция, которая для каждого значения лага выдает коэффициент корреляции между временным рядом и временным рядом с эти лагом.
    * Частная автокорреляционная функция - функция, которая для каждого лага k выдает автокорреляцию между исходным рядом $z_{t}$, очиценным от линейной связи со всеми предшествующими лагами от $z_{t+1}$ до $z_{t+k-1}$ и рядом  $z_{{t+k}}$. 

2. Какой временной ряд называется стационарным? Может ли временной ряд с трендом быть стационарным? А с циклом?

    * Стационарный ряд - такой ряд, у которого вероятностные характеристики не меняются во времени.

    * Если говорим про тренд - мы будем ожидать, что такой ряд интегрированный порядка 1, и сам не стационарный. Но ряд из растностей может быть стационарным, а может и не быть.

    * Если мы говорим про циклы, имеющие некий фиксированный период (сезонность), то такой ряд не стационарный, если длина цикла случайна, то такой ряд может быть стационарным.
  

3. Какую гипотезу проверяет тест Дики-Фуллера.

    * Тесть проверяет гипотезу о стационарности ряда и $H_0$ - ряд не стационарный, то есть когда мы отвергаем гипотезу, мы узнаем, что ряд стационарный.

4. Какой временной рядм можно описать хорошо с помощью модели ARMA?

    * Под эту модель подходят хорошо ряды со следующими свойствами:

        * Ряд стационарный.
        * Нет внешних переменных, которые влияют на значения ряда (сходу без модификации в эту модель не добавить регрессую от других рядов).
        * Есть зависимость от значений переменной в прошлом и от остатков на прошлых шагах.

5. На что нужно обращать внимание при анализе остатков модели

    * Проверять соответствие их распределения нормальному.
    * Тестировать их на стационарность.
    * Обращать внимание на наличие периодичности и/или гетероскедастичности.



# Задание 2
Шампунь, продажи за 3 года. Подобрать модель SARIMA и сделать прогноз на 3 месяца вперед.

In [3]:
shampoo = pd.read_csv('data/shampoo.csv')


In [4]:
shampoo["Month"] = pd.to_datetime('0'+shampoo.Month, format='%y-%m')
shampoo = shampoo.set_index("Month")
shampoo.head()

,Sales
Month,
2001-01-01,266.0
2001-02-01,145.9
2001-03-01,183.1
2001-04-01,119.3
2001-05-01,180.3


In [5]:
decompostition = sm.tsa.seasonal_decompose(shampoo, model= 'additive')
fig = decompostition.plot()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
plt.figure()
lag_plot(shampoo)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Видим, что есть тренд, и сезонность, ожидание, что ряд не стационарный и что есть автокорреляция. Проверим:

In [7]:
from statsmodels.tsa.stattools import adfuller


def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

In [8]:
print("Критерий Дики-Фуллера для начального ряда: pvalue =",sm.tsa.stattools.adfuller(shampoo.Sales)[1])

Критерий Дики-Фуллера для начального ряда: pvalue = 1.0


In [9]:
adf_test(shampoo)

Results of Dickey-Fuller Test:
Test Statistic                  3.060142
p-value                         1.000000
#Lags Used                     10.000000
Number of Observations Used    25.000000
Critical Value (1%)            -3.723863
Critical Value (5%)            -2.986489
Critical Value (10%)           -2.632800
dtype: float64


Такой ряд не стационарный - перейдем к разностям:

In [10]:
shampoo['sales_boxcox'], lmbd = stats.boxcox(shampoo['Sales'])
shampoo['sales_boxcox_diff'] = shampoo.sales_boxcox - shampoo.sales_boxcox.shift(1)
sm.tsa.seasonal_decompose(shampoo.sales_boxcox_diff[1:]).plot()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
adf_test(shampoo.sales_boxcox_diff[1:])

Results of Dickey-Fuller Test:
Test Statistic                 -3.971330
p-value                         0.001568
#Lags Used                      9.000000
Number of Observations Used    25.000000
Critical Value (1%)            -3.723863
Critical Value (5%)            -2.986489
Critical Value (10%)           -2.632800
dtype: float64


In [12]:
shampoo['sales_diff'] = shampoo.Sales - shampoo.Sales.shift(1)
sm.tsa.seasonal_decompose(shampoo.sales_diff[1:]).plot()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
adf_test(shampoo.sales_diff[1:])

Results of Dickey-Fuller Test:
Test Statistic                -7.249074e+00
p-value                        1.799857e-10
#Lags Used                     1.000000e+00
Number of Observations Used    3.300000e+01
Critical Value (1%)           -3.646135e+00
Critical Value (5%)           -2.954127e+00
Critical Value (10%)          -2.615968e+00
dtype: float64


без преобразования Бокса-Кокса, мы получаем более низкое p-value для теста Дики-Фулера, поэтому возьмем только разницы, а чтобы брать дельты дополнительные, уже нет данных

посмотрим на автокорреляционные и частные автокорреляционные графики

In [14]:
fig, (ax1, ax2) = plt.subplots(nrows=2,figsize=(15,8)) 

sm.graphics.tsa.plot_acf(shampoo.sales_diff[1:].values.squeeze(), lags=15, ax=ax1)
sm.graphics.tsa.plot_pacf(shampoo.sales_diff[1:].values.squeeze(), lags=15, ax=ax2)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Согласно этим графикам, в качестве начальных значений необходимо взять: $Q=1$, $q=1$, $P=1$, $p=1$.

In [15]:
from itertools import product # произведения множеств

# создадим список из значений параметров для перебора моделей


d = 1
D = range(0, 2)
ps = range(0, 2)
qs = range(0, 2)
Ps = range(0, 2)
Qs = range(0, 2)
trend = ['n', 'c', 't', 'ct']

parameters = product(ps ,qs, Ps, Qs, D, trend)
parameters_list = list(parameters)

In [16]:
from tqdm.notebook import tqdm


In [17]:
results = [] # здесь будем хранить результаты
warnings.filterwarnings('ignore') # не для всех параметров модели будут обучаться; отключим предупреждения

for param in tqdm(parameters_list):
    #try-except нужен, чтобы обработать случаи, когда модель не обучается
    try:
        model=sm.tsa.statespace.SARIMAX(shampoo.Sales, order=(param[0], d, param[1]), 
                                        seasonal_order=(param[2], param[4], param[3], 12), trend=param[5]).fit(disp=-1)
    
    #выводим параметры, на которых модель не обучается и переходим к следующему набору
    except ValueError:
        print('Wrong parameters:', param)
        continue
    
    results.append([param, model.aic])
# преобразуем результаты в DataFrame и добавим названия столбцов

result_table = pd.DataFrame(results)
result_table.columns = ['Parameters', 'AIC']

  0%|          | 0/128 [00:00<?, ?it/s]

In [18]:
result_table.sort_values(by = 'AIC', ascending=True).head()

,Parameters,AIC
117,"(1, 1, 1, 0, 1, c)",275.535467
109,"(1, 1, 0, 1, 1, c)",275.935619
118,"(1, 1, 1, 0, 1, t)",276.586929
119,"(1, 1, 1, 0, 1, ct)",277.122648
125,"(1, 1, 1, 1, 1, c)",277.258594


Лучшей моделью получилась SARIMA (1,1,1)x(1,0,1).

In [19]:
best_model=sm.tsa.statespace.SARIMAX(shampoo.Sales, order=(1,1,1), trend='c', seasonal_order=(1,1,0,12)).fit(disp=-1)
print(best_model.summary())

                                      SARIMAX Results                                      
Dep. Variable:                               Sales   No. Observations:                   36
Model:             SARIMAX(1, 1, 1)x(1, 1, [], 12)   Log Likelihood                -132.768
Date:                             Sat, 07 Aug 2021   AIC                            275.535
Time:                                     18:49:38   BIC                            281.213
Sample:                                 01-01-2001   HQIC                           276.963
                                      - 12-01-2003                                         
Covariance Type:                               opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     27.5420      4.660      5.910      0.000      18.408      36.676
ar.L1         -0.6273      

Посмотрим на остатки

In [20]:
best_model.resid[13:].plot()
plt.ylabel('Residuals')
plt.show()

**Несмещенность.** Проверим несмещенность с помощью критерия знаков (самый универсальный критерий).

In [21]:
from statsmodels.stats.descriptivestats import sign_test
print("Критерий знаков для остатков: pvalue =",sign_test(best_model.resid[13:])[1])

Критерий знаков для остатков: pvalue = 0.21003961563110352


**Стационарность.** Проверим стационарность с помщью критерия Дики-Фуллера.

In [22]:
print("Критерий Дики-Фуллера для остатков: pvalue =",sm.tsa.stattools.adfuller(best_model.resid[13:])[1])

Критерий Дики-Фуллера для остатков: pvalue = 0.000610127398725332


**Неавтокоррелированность.** Чтобы определить независимость остаткок, посмотрим на график автокорреляционной функции.

In [23]:
sm.graphics.tsa.plot_acf(best_model.resid[13:].values.squeeze(), lags=15)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Еще и по Jarque-Bera не отвергается гипотеза о нормальности

In [24]:
figure = best_model.plot_diagnostics()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
shampoo['model'] = best_model.fittedvalues

In [28]:
fig, ax = plt.subplots()
ax.plot(shampoo.Sales, label='actual')
ax.plot(shampoo.model, color='r', label='prediction')
ax.plot(best_model.get_forecast(3).predicted_mean, label='forecast')
ax.plot(best_model.get_forecast(3).conf_int()['lower Sales'], label='lower_conf', color = 'gray')
ax.plot(best_model.get_forecast(3).conf_int()['upper Sales'], label='upper_conf', color = 'gray')
ax.fill_between(best_model.get_forecast(3).predicted_mean.index, 
                best_model.get_forecast(3).conf_int()['lower Sales'], 
                best_model.get_forecast(3).conf_int()['upper Sales'], facecolor='lightgray', alpha=0.5)
ax.legend()

ax.set_ylabel('volume')
ax.set_title('Shampoo sales')
fig.tight_layout()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Мы получили прогноз продаж шампуня, сравнили его с историей и построили прогноз в будущее с доверительным интервалом

# Задание 3
Аналогично про новорожденных девочек


In [35]:
girls = pd.read_csv('data/female-births.csv') 
girls['Date'] = pd.to_datetime(girls['Date'])
girls = girls.set_index("Date")
girls.head()

,Births
Date,
1959-01-01,35
1959-01-02,32
1959-01-03,30
1959-01-04,31
1959-01-05,44


In [36]:
decompostition = sm.tsa.seasonal_decompose(girls, model= 'additive')
fig = decompostition.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
plt.figure()
lag_plot(girls)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
adf_test(girls)

Results of Dickey-Fuller Test:
Test Statistic                  -4.808291
p-value                          0.000052
#Lags Used                       6.000000
Number of Observations Used    358.000000
Critical Value (1%)             -3.448749
Critical Value (5%)             -2.869647
Critical Value (10%)            -2.571089
dtype: float64


Есть небольшая зависимость значения от вчерашнего значения, но сам исходный ряд уже является стационарным

In [46]:
girls['births_boxcox'], lmbd = stats.boxcox(girls['Births'])
girls['births_boxcox_diff'] = girls.births_boxcox - girls.births_boxcox.shift(1)
sm.tsa.seasonal_decompose(girls.births_boxcox_diff[1:]).plot()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [47]:
adf_test(girls.births_boxcox_diff[1:])

Results of Dickey-Fuller Test:
Test Statistic                -1.285895e+01
p-value                        5.159404e-24
#Lags Used                     6.000000e+00
Number of Observations Used    3.570000e+02
Critical Value (1%)           -3.448801e+00
Critical Value (5%)           -2.869670e+00
Critical Value (10%)          -2.571101e+00
dtype: float64


В таком варианте мы полностью избавились от переодичности (сделали преобразования данных Бокса-Кокса и перешли к разинцам)

Посмотрим на графики автокорреляции и частной автокорреляции

In [51]:
fig, (ax1, ax2) = plt.subplots(nrows=2,figsize=(15,8)) 

sm.graphics.tsa.plot_acf(girls.births_boxcox_diff[1:].values.squeeze(), lags=31, ax=ax1)
sm.graphics.tsa.plot_pacf(girls.births_boxcox_diff[1:].values.squeeze(), lags=31, ax=ax2)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Согласно этим графикам, в качестве начальных значений необходимо взять: $Q=3$, $q=7$, $P=1$, $p=1$.

In [56]:
# создадим список из значений параметров для перебора моделей
d = 1
D = 1
ps = range(0, 2)
qs = range(0, 8)
Ps = range(0, 2)
Qs = range(0, 4)
trend = ['n', 'c', 't', 'ct']

parameters = product(ps ,qs, Ps, Qs, trend)
parameters_list = list(parameters)

In [79]:
results2 = [] # здесь будем хранить результаты
warnings.filterwarnings('ignore') # не для всех параметров модели будут обучаться; отключим предупреждения

for param in tqdm(parameters_list):
    #try-except нужен, чтобы обработать случаи, когда модель не обучается
    try:
        model=sm.tsa.statespace.SARIMAX(girls.births_boxcox, order=(param[0], d, param[1]), 
                                        seasonal_order=(param[2], D, param[3], 7), trend=param[4]).fit(disp=-1)
    
    #выводим параметры, на которых модель не обучается и переходим к следующему набору
    except ValueError:
        print('Wrong parameters:', param)
        continue
    
    results2.append([param, model.aic])
# преобразуем результаты в DataFrame и добавим названия столбцов

result_table2 = pd.DataFrame(results2)
result_table2.columns = ['Parameters', 'AIC']

  0%|          | 0/512 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [60]:
result_table2.sort_values(by = 'AIC', ascending=True).head()


,Parameters,AIC
320,"(1, 2, 1, 2, n)",2428.249635
288,"(1, 1, 1, 2, n)",2428.320040
88,"(0, 2, 1, 2, n)",2428.695778
120,"(0, 3, 1, 2, n)",2429.447720
56,"(0, 1, 1, 2, n)",2429.467435


In [ ]:
Лучшей моделью получилась SARIMA (1,1,2)x(1,1,2).

In [62]:
best_model2=sm.tsa.statespace.SARIMAX(girls.Births, order=(1,1,2), trend='n', seasonal_order=(1,1,2,7)).fit(disp=-1)
print(best_model2.summary())

                                     SARIMAX Results                                     
Dep. Variable:                            Births   No. Observations:                  365
Model:             SARIMAX(1, 1, 2)x(1, 1, 2, 7)   Log Likelihood               -1207.125
Date:                           Sat, 07 Aug 2021   AIC                           2428.250
Time:                                   19:58:55   BIC                           2455.394
Sample:                               01-01-1959   HQIC                          2439.046
                                    - 12-31-1959                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.6793      0.287      2.369      0.018       0.117       1.241
ma.L1         -1.5431      0.320     -4.822

**Несмещенность.** Проверим несмещенность с помощью критерия знаков (самый универсальный критерий).


In [65]:
print("Критерий знаков для остатков: pvalue =",sign_test(best_model2.resid[13:])[1])


Критерий знаков для остатков: pvalue = 0.016340436317813394


**Стационарность.** Проверим стационарность с помщью критерия Дики-Фуллера.


In [66]:
print("Критерий Дики-Фуллера для остатков: pvalue =",sm.tsa.stattools.adfuller(best_model2.resid[13:])[1])


Критерий Дики-Фуллера для остатков: pvalue = 2.092616915145717e-30


**Неавтокоррелированность.** Чтобы определить независимость остаткок, посмотрим на график автокорреляционной функции.


In [67]:
sm.graphics.tsa.plot_acf(best_model2.resid[13:].values.squeeze(), lags=15)
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Еще и по Jarque-Bera не отвергается гипотеза о нормальности


In [68]:
figure = best_model2.plot_diagnostics()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [70]:
best_model2.get_forecast(31).conf_int()

,lower Births,upper Births
1960-01-01,31.820696,58.865920
1960-01-02,30.401909,57.697331
1960-01-03,28.007258,55.445230
1960-01-04,32.620898,60.145795
1960-01-05,32.219036,59.800985
1960-01-06,33.907507,61.529831
1960-01-07,32.809002,60.462036
1960-01-08,30.431369,58.512237
1960-01-09,30.353791,58.479601
1960-01-10,27.582619,55.743583


In [77]:
girls['model'] = best_model2.fittedvalues
fig, ax = plt.subplots()
ax.plot(girls.Births, label='actual')
ax.plot(girls.model[15:], color='r', label='prediction')
ax.plot(best_model2.get_forecast(31).predicted_mean, label='forecast')
ax.plot(best_model2.get_forecast(31).conf_int()['lower Births'], label='lower_conf', color = 'gray')
ax.plot(best_model2.get_forecast(31).conf_int()['upper Births'], label='upper_conf', color = 'gray')
ax.fill_between(best_model2.get_forecast(31).predicted_mean.index, 
                best_model2.get_forecast(31).conf_int()['lower Births'], 
                best_model2.get_forecast(31).conf_int()['upper Births'], facecolor='lightgray', alpha=0.5)
ax.legend()

ax.set_ylabel('count')
ax.set_title('births')
fig.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …